<a href="https://colab.research.google.com/github/Jamalun/Data_Capstone_Design/blob/master/BERT_Sentimental_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 불러오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 학습용 데이터셋 불러오기
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/Colab/Kaggle_translated.xlsx', names = ["sentiment","english", "korean" ])
df.head()

,sentiment,english,korean
0,neutral,Technopolis plans to develop in stages an area...,"테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 100,0..."
1,negative,The international electronic industry company ...,국제적인 전자 산업 회사 Elcoteq는 그것의 Talinn 시설로부터 10명의 직...
2,positive,With the new production plant the company woul...,새로운 생산 공장과 함께 회사는 수요의 예상되는 증가를 충족시키기 위하여 그것의 능...
3,positive,According to the company 's updated strategy f...,"2009-2012년 회사의 최신 전략에 따르면, Basware는 순매출의 10%-2..."
4,positive,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...,ASPOCOMP의 파이낸싱은 기술적으로 더 까다로운 HDI 인쇄 회로 기판 PCB에...


# 데이터 라벨링

In [3]:
import numpy as np

In [4]:
labels = []
for sentiment in df['sentiment']:
  if (sentiment == 'neutral'):
    labels.append(2)
  if (sentiment == 'negative'):
    labels.append(0)
  if (sentiment == 'positive'):
    labels.append(1)

df['label'] = labels
  

In [5]:
# 라벨링이 0,1,2로 잘 되었음을 확인
df['label'].unique()

array([2, 0, 1])

In [6]:
# 중립은 제외하고 이진 분류하기//  
'''
index1 = df[df['label'] == 2].index
df.drop(index1, inplace = True)
df['label'].unique()
'''

"\nindex1 = df[df['label'] == 2].index\ndf.drop(index1, inplace = True)\ndf['label'].unique()\n"

In [7]:
# Train / Test set 분리
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, random_state=42)
print("train shape is:", len(train))
print("test shape is:", len(test))

train shape is: 3876
test shape is: 969


## BERT 시도해보기 !

In [8]:
# Hugging Face의 트랜스포머 모델을 설치
!pip install transformers

     |████████████████████████████████| 3.1 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 46.8 MB/s 
     |████████████████████████████████| 895 kB 48.6 MB/s 
     |████████████████████████████████| 3.3 MB 36.3 MB/s 
     |████████████████████████████████| 59 kB 6.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [9]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [10]:
# 문장 추출
sentences = train['korean']
sentences[:10]

1215    EQT는 스웨덴 투자 그룹 Nordstjernan에게 약 3천 5백만 (4천 7백만...
432     2010년 9월, 핀란드 그룹은 덴마크 회사인 로즈 가금류 A-S를 현금과 주식의 ...
4339    현재의 기준 등급 기구들은 높은 가격과 일상적인 사용을 위한 복잡성 때문에 작동하기...
289     그 회사는 새로운 함대를 건조하는 과정에 있으며 65,200 드워트의 P-Max 유...
2588       각각 약 40개의 콘도와 함께, 건물들은 4 또는 5개의 단계에서 지어질 것입니다.
4120    노키아와 캡콤은 레지던트 Evil Degeneration이 올해 말에 N-Gage로...
4740              9월에만, 그 시장은 1,928만 리터로 전년대비 10.2% 감소했다.
535     이 구독으로, Fortum의 TGK-10의 소유권은 주식과 의결권의 76%를 약간 ...
1479                      "유연성은 행복한 대출자들에게 지불할 작은 대가이다. "
188     주식 비율은 2007년 3분기에 54.2 퍼센트에 비해 60.9 퍼센트였습니다, 냉...
Name: korean, dtype: object

In [11]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] EQT는 스웨덴 투자 그룹 Nordstjernan에게 약 3천 5백만 (4천 7백만 달러)에 그것의 잔여 지분을 매각함으로써 휴대폰 충전기를 만드는 핀란드 회사인 Salcomp로부터 그것의 퇴장을 완료했다. [SEP]',
 '[CLS] 2010년 9월, 핀란드 그룹은 덴마크 회사인 로즈 가금류 A-S를 현금과 주식의 조합으로 23.9 m에 사기로 합의했다. [SEP]',
 '[CLS] 현재의 기준 등급 기구들은 높은 가격과 일상적인 사용을 위한 복잡성 때문에 작동하기가 어렵다. [SEP]',
 '[CLS] 그 회사는 새로운 함대를 건조하는 과정에 있으며 65,200 드워트의 P-Max 유조선 10척을 발주했다. [SEP]',
 '[CLS] 각각 약 40개의 콘도와 함께, 건물들은 4 또는 5개의 단계에서 지어질 것입니다. [SEP]',
 '[CLS] 노키아와 캡콤은 레지던트 Evil Degeneration이 올해 말에 N-Gage로 출시될 것이라고 발표했다. [SEP]',
 '[CLS] 9월에만, 그 시장은 1,928만 리터로 전년대비 10.2% 감소했다. [SEP]',
 '[CLS] 이 구독으로, Fortum의 TGK-10의 소유권은 주식과 의결권의 76%를 약간 넘는 수준으로 증가했다. [SEP]',
 '[CLS] "유연성은 행복한 대출자들에게 지불할 작은 대가이다. " [SEP]',
 '[CLS] 주식 비율은 2007년 3분기에 54.2 퍼센트에 비해 60.9 퍼센트였습니다, 냉동 식품 사업의 순매출은 총 11.0 유로였습니다, 2006년 3분기부터 약 5% 증가하였습니다. [SEP]']

In [12]:
# 라벨 추출
labels = train['label'].values
labels

array([2, 1, 0, ..., 2, 2, 1])

In [13]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

[CLS] EQT는 스웨덴 투자 그룹 Nordstjernan에게 약 3천 5백만 (4천 7백만 달러)에 그것의 잔여 지분을 매각함으로써 휴대폰 충전기를 만드는 핀란드 회사인 Salcomp로부터 그것의 퇴장을 완료했다. [SEP]
['[CLS]', 'E', '##Q', '##T', '##는', '스', '##웨', '##덴', '투', '##자', '그', '##룹', 'Nord', '##st', '##jer', '##nan', '##에게', '약', '3', '##천', '5', '##백', '##만', '(', '4', '##천', '7', '##백', '##만', '달', '##러', ')', '에', '그', '##것', '##의', '잔', '##여', '지', '##분을', '매', '##각', '##함', '##으로', '##써', '휴', '##대', '##폰', '충', '##전', '##기를', '만', '##드는', '핀', '##란드', '회', '##사', '##인', 'Sal', '##com', '##p', '##로부터', '그', '##것', '##의', '퇴', '##장을', '완', '##료', '##했다', '.', '[SEP]']


In [14]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,    142,  19282,  11090,  11018,   9477, 119173, 118790,
         9881,  13764,   8924,  87114,  12004,  10562,  17792,  13470,
        26212,   9539,    124,  38631,    126,  63218,  19105,    113,
          125,  38631,    128,  63218,  19105,   9061,  30873,    114,
         9559,   8924, 118627,  10459,   9653,  29935,   9706,  97005,
         9258,  66540,  48533,  11467,  73131,  10013,  14423, 119401,
         9770,  16617,  29669,   9248,  61945,   9948,  61592,   9998,
        12945,  12030,  64831,  22530,  10410,  92515,   8924, 118627,
        10459,   9880,  35963,   9591,  38688,  12490,    119,    102,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [15]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [16]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=2018, 
                                                                                    test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2018, 
                                                       test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)				

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

tensor([  101,  9959, 19105,  9485, 31928, 10459,  9744, 12692, 44321, 10892,
        10258, 19105,   157, 11259, 12022, 12638,   128,   119,   126, 35198,
         9875, 10459,  9641, 30134,  9993, 29364, 58303, 48345,   119,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [17]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [18]:
# 문장 추출
sentences = test['korean']
sentences[:10]

3206    그 회사는 러시아에서 Tomsk 지역에 베니어 공장에게 기계를 배달하기로 되어 있었...
1684               UPM-Kymmene은 4분기 연속 긍정적인 현금 유동성을 창출했다.
1044    방법에서 빙어 스푸트 2는 빙어 스푸트에 대하여 이동 가능한 10개의 차폐벽에 의해...
4144    그러나 누프는 핀란드 관광객들이 산 맥주도 포함된다고 상기시키면서 "유럽 규모로는 ...
1538              Penttil+E의 시각에서, 거래의 가장 중요한 이유는 러시아입니다.
1034       FinancialWire tm은 완전히 독립적이고 독점적인 뉴스 유선 서비스입니다.
2107    4월에 만들어진 지불에 이어, 그 회사는 금융 기관들로부터 대출에 총 230만 유로...
4256    Innova 2 프로젝트의 총 면적은 약 10,000 평방 미터 (107,600 평...
4020    핀란드의 가정 장식 및 예술 공예 소매상 Tiimari Plc (OMX Helski...
3568      납품은 2007년 하반기에 시작될 예정이며 공장 가동은 2008년으로 예정되어 있다.
Name: korean, dtype: object

In [19]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] 그 회사는 러시아에서 Tomsk 지역에 베니어 공장에게 기계를 배달하기로 되어 있었습니다. [SEP]',
 '[CLS] UPM-Kymmene은 4분기 연속 긍정적인 현금 유동성을 창출했다. [SEP]',
 '[CLS] 방법에서 빙어 스푸트 2는 빙어 스푸트에 대하여 이동 가능한 10개의 차폐벽에 의해 작업 영역 6에서 분리된다. [SEP]',
 '[CLS] 그러나 누프는 핀란드 관광객들이 산 맥주도 포함된다고 상기시키면서 "유럽 규모로는 매우 높은 수치이다."라고 말했다. [SEP]',
 '[CLS] Penttil+E의 시각에서, 거래의 가장 중요한 이유는 러시아입니다. [SEP]',
 '[CLS] FinancialWire tm은 완전히 독립적이고 독점적인 뉴스 유선 서비스입니다. [SEP]',
 '[CLS] 4월에 만들어진 지불에 이어, 그 회사는 금융 기관들로부터 대출에 총 230만 유로를 가지고 있습니다. [SEP]',
 '[CLS] Innova 2 프로젝트의 총 면적은 약 10,000 평방 미터 (107,600 평방 피트)가 될 것입니다. [SEP]',
 '[CLS] 핀란드의 가정 장식 및 예술 공예 소매상 Tiimari Plc (OMX Helski : TII1V)는 8월 18일 월요일 2008년 1월부터 6월까지의 순 판매에서 EUR3.3m의 영업 손실을 보고했다. [SEP]',
 '[CLS] 납품은 2007년 하반기에 시작될 예정이며 공장 가동은 2008년으로 예정되어 있다. [SEP]']

In [20]:
# 라벨 추출
labels = test['label'].values
labels

array([2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 0, 2, 1, 2, 1, 1, 0, 0, 2,
       1, 1, 2, 1, 2, 1, 1, 0, 2, 2, 1, 0, 2, 1, 0, 1, 2, 2, 1, 2, 0, 2,
       1, 2, 2, 2, 1, 1, 0, 2, 1, 0, 2, 1, 2, 2, 2, 2, 1, 1, 1, 2, 0, 2,
       1, 2, 2, 2, 2, 1, 1, 1, 1, 2, 1, 2, 2, 1, 2, 0, 1, 2, 1, 1, 1, 1,
       1, 1, 1, 2, 0, 0, 2, 1, 2, 2, 1, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2,
       2, 1, 0, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 1, 2, 0, 1,
       2, 2, 1, 2, 1, 0, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 0, 0, 1, 1, 2, 1, 2, 1, 2, 0, 2, 2, 2, 1, 2, 2, 1, 2,
       2, 2, 2, 2, 1, 1, 2, 1, 2, 0, 2, 0, 2, 0, 2, 2, 1, 1, 2, 1, 0, 2,
       0, 1, 2, 2, 1, 0, 2, 1, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2,
       1, 2, 2, 2, 0, 2, 2, 1, 2, 2, 2, 1, 1, 2, 0, 1, 2, 1, 2, 2, 2, 2,
       2, 0, 1, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1,
       0, 2, 0, 2, 2, 1, 2, 2, 0, 1, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 1, 2,
       2, 2, 0, 2, 1, 2, 1, 0, 0, 2, 2, 2, 1, 2, 1,

In [21]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] 그 회사는 러시아에서 Tomsk 지역에 베니어 공장에게 기계를 배달하기로 되어 있었습니다. [SEP]
['[CLS]', '그', '회', '##사는', '러시아', '##에서', 'Tom', '##sk', '지역', '##에', '베', '##니', '##어', '공', '##장에', '##게', '기', '##계를', '배', '##달', '##하기', '##로', '되어', '있', '##었', '##습', '##니다', '.', '[SEP]']


In [22]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,   8924,   9998,  64379,  62834,  11489,  12275,  11478,
        58939,  10530,   9344,  25503,  12965,   8896,  85903,  14153,
         8932,  54163,   9330,  89851,  22440,  11261,  37909,   9647,
       119138, 119081,  48345,    119,    102,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [23]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [24]:
# 데이터를 파이토치의 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

tensor([   101,   8924,   9998,  64379,  62834,  11489,  12275,  11478,  58939,
         10530,   9344,  25503,  12965,   8896,  85903,  14153,   8932,  54163,
          9330,  89851,  22440,  11261,  37909,   9647, 119138, 119081,  48345,
           119,    102,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0, 

In [25]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [26]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [27]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [28]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=3)  ## num_labels = 3
model.cuda()

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [29]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 4

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 처음에 학습률을 조금씩 변화시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [30]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [31]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [32]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...

  Average training loss: 0.80
  Training epcoh took: 0:12:11

Running Validation...
  Accuracy: 0.77
  Validation took: 0:00:26

======== Epoch 2 / 4 ========
Training...

  Average training loss: 0.51
  Training epcoh took: 0:12:11

Running Validation...
  Accuracy: 0.79
  Validation took: 0:00:26

======== Epoch 3 / 4 ========
Training...

  Average training loss: 0.36
  Training epcoh took: 0:12:11

Running Validation...
  Accuracy: 0.81
  Validation took: 0:00:26

======== Epoch 4 / 4 ========
Training...

  Average training loss: 0.27
  Training epcoh took: 0:12:11

Running Validation...
  Accuracy: 0.79
  Validation took: 0:00:26

Training complete!


In [33]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))


Accuracy: 0.79
Test took: 0:01:05
